# Solving with CAVI

In [ ]:
import numpy as np
from scipy.stats import norm, gamma
import cvxpy as cp
from typing import Callable, List
from cavi import cavi
from elbo import digamma_approx, elbo, f_0, f_1

## Set up Bayesian Regression Model

In [ ]:
# Parameters (# of samples, # of features, beta params, alpha params)
n = 100
p = 5
lambda_ = 10
a0 = 10.0  
b0 = 15.0  

# Simulate X
X = np.random.randn(n, p)  

# Simulate alpha 
alpha = gamma.rvs(1, loc = a0, scale = 1 / b0)

# Simulate beta 
beta = np.random.normal(0, np.sqrt(1 / lambda_), p)

# Simulate y
y = np.random.normal(X @ beta, 1 / np.sqrt(alpha))

In [ ]:
# Define variables
a = cp.Variable(value=12, nonneg=True)
c = cp.Variable(value=800, nonneg=True)
d = cp.Variable(value=51, nonneg=True)
sigma = cp.Variable((p, p), symmetric=True)
mu = cp.Variable(p, nonneg=True)

# set constraints 
constraints = []
constraints.append(sigma >> 0)
constraints.append(mu >= 0)
constraints.append(a >= 0)
constraints.append(c >= 0)
constraints.append(d >= 0)

In [1]:
# run cavi
initial_params = [a, c, d, mu, sigma]
params, history = cavi(initial_params, elbo, maximize=True, max_iter=50, tol=1e-5)

---------------------------------------------------------------------------
DCPError                                  Traceback (most recent call last)
Cell In[5], line 3
      1 # run cavi
      2 initial_params = [a, c, d, mu, sigma]
----> 3 params, history = cavi(initial_params, elbo, maximize=True, max_iter=50, tol=1e-5)

File ~/GitHub/ee-227bt-final-project/cavi.py:71, in cavi(initial_params, objective_fn, maximize, max_iter, tol)
     69 obj_expr = objective_fn(*current_q_vars)
     70 problem = cp.Problem(cp.Minimize(obj_expr))
---> 71 problem.solve(solver=cp.SCS, verbose=False)
     73 # Update parameter
     74 if q_i_var.value is not None:

File /opt/homebrew/lib/python3.11/site-packages/cvxpy/problems/problem.py:577, in Problem.solve(self, *args, **kwargs)
    574         raise ValueError(
    575             "Cannot specify both 'solver' and 'solver_path'. Please choose one.")
    576     return self._solve_solver_path(solve_func,solver_path, args, kwargs)
--> 577 return so